In [1]:
import sys
print(sys.executable)

/home/rs/PycharmProjects/snake-multi/.venv/bin/python3.10


In [2]:
import os
os.chdir("../..")
os.getcwd()

'/home/rs/PycharmProjects/snake-multi'

In [3]:
import random
import logging
import sys
from collections import deque
from functools import partial
from math import floor, ceil
from pathlib import Path

import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from src.simulation.simulation import Simulation, Vector2
from src.simulation.board_generator import make_simple_board

from src.model.model_utils import get_model_moves, get_model_scores, get_rotated_state, ReplayBuffer

tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[0], True)

2025-10-13 23:47:18.788777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760392038.799995   18549 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760392038.803412   18549 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760392038.813042   18549 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760392038.813052   18549 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760392038.813054   18549 computation_placer.cc:177] computation placer alr

In [4]:
view_range = 15

replay_buffer_size = 50_000
replay_buffer_threshold_multiplier = 1.3

discount_factor = 0.9

batch_size = 32
loss_fn = tf.keras.losses.mse
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)

n_episodes = 2_000

best_model_window = 7

I0000 00:00:1760392040.546391   18549 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1779 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1


In [5]:
def calculate_score(simulation: Simulation, ate_food: bool, died: bool, n_snakes_killed: int, food_progress: int):
    if died:
        return -10

    reward = 0
    
    if ate_food:
        reward += 10

    reward += n_snakes_killed * 10

    if reward == 0:
        reward += food_progress

    return reward

In [6]:
Dense = partial(tf.keras.layers.Dense, activation="relu", kernel_initializer="he_normal")
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal")

online_model = tf.keras.Sequential([
    tf.keras.layers.Input((15, 15)),
    tf.keras.layers.CategoryEncoding(num_tokens=7, output_mode="one_hot"),
    DefaultConv2D(filters=32, kernel_size=7),
    DefaultConv2D(filters=64, kernel_size=5),
    DefaultConv2D(filters=64, kernel_size=3),
    tf.keras.layers.MaxPool2D(pool_size=3),
    DefaultConv2D(filters=128, kernel_size=3),
    DefaultConv2D(filters=128, kernel_size=3),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    Dense(32, activation="relu", kernel_initializer="he_normal"),
    Dense(32, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1)
])

target_model = tf.keras.models.clone_model(online_model)
target_model.set_weights(online_model.get_weights())

view_type = "full"

In [7]:
def fix_float(val: float) -> str:
    return str(round(float(val), 8)).replace('.', '_')

model_name = f"r{view_range}_{view_type}_rb_{fix_float(replay_buffer_threshold_multiplier)}_e_{n_episodes}_lr_{fix_float(optimizer.learning_rate)}"

model_name += "_timeout_conv_battle_food_progress"

In [8]:
for i in range(100):
    file_path = Path(f"logs_{model_name}_v{i}.log")

    if not file_path.is_file():
        model_name += f"_v{i}"
        break
else:
    raise RuntimeError("If it's not a bug, train less models")

logging.basicConfig(level=logging.DEBUG)

log_formatter = logging.Formatter("%(message)s")

std_handler = logging.StreamHandler(sys.stdout)
std_handler.setLevel(logging.INFO)
std_handler.setFormatter(log_formatter)

file_handler = logging.FileHandler(file_path)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(log_formatter)

logger = logging.getLogger()
logger.handlers.clear()
logger.addHandler(std_handler)
logger.addHandler(file_handler)

In [9]:
online_model.summary(print_fn=logger.info)
online_model.summary()

Model: "sequential"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ category_encoding               │ (None, 15, 15, 7)      │             0 │
│ (CategoryEncoding)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 15, 15, 32)     │        11,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 15, 15, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 15, 15, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 5, 5, 64)    

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ category_encoding               │ (None, 15, 15, 7)      │             0 │
│ (CategoryEncoding)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 15, 15, 32)     │        11,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 15, 15, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 15, 15, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 5, 5, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 5, 5, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 338,145 (1.29 MB)

 Trainable params: 338,145 (1.29 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Make sure model takes correct input
simulation = Simulation(calculate_score, False)
simulation.reset(make_simple_board(np.array([15, 15])), 2, 1, 1)

online_model.predict(np.array([simulation.get_snake_view(0, view_type, view_range)]))

I0000 00:00:1760392041.170863   18614 service.cc:152] XLA service 0x7fbca0003c40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760392041.170892   18614 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 3GB, Compute Capability 6.1
2025-10-13 23:47:21.177349: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1760392041.202883   18614 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-10-13 23:47:21.394502: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.16 = (f32[1,64,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,32,15,15]{3,2,1,0} %bitcast.329, f32[64,32,5,5]{3,2,1,0} %bitcast.336, f32[64]{0} %bitcast.338), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudn

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 668ms/step


I0000 00:00:1760392041.769927   18614 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


array([[0.459063]], dtype=float32)

In [11]:
def epsilon_greedy(snakes_states: list[np.ndarray], snakes_possible_moves: list[list[Vector2]], snakes_alive: list[bool], epsilon: float=0.):
    n_snakes = len(snakes_states)

    process_mask = [True for _ in range(n_snakes)]
    
    for i, alive in enumerate(snakes_alive):
        if not alive or np.random.rand() < epsilon:
            process_mask[i] = False

    moves = get_model_moves(online_model, snakes_states, snakes_possible_moves, process_mask, softmax=True)

    for i, move in enumerate(moves):
        if move == (0, 0):
            moves[i] = random.choice(simulation.get_legal_moves(i))

    return moves

In [12]:
#priority_replay_buffer = deque(maxlen=replay_buffer_size // 2)
#replay_buffer = deque(maxlen=replay_buffer_size // 2)

replay_buffer = ReplayBuffer(
    priority_size=replay_buffer_size // 2,
    normal_size=replay_buffer_size // 2,
    threshold_multiplier=replay_buffer_threshold_multiplier
)

In [13]:
def play_one_step(simulation: Simulation, states_before: list[np.ndarray], possible_moves_before: list[Vector2], snakes_alive_before: list[bool], epsilon: float):
    # Save states
    moves = epsilon_greedy(states_before, possible_moves_before, snakes_alive_before, epsilon)

    scores, running = simulation.next(moves)

    states_after = [simulation.get_snake_view(i, view_type, view_range) for i in range(simulation.n_snakes)]
    possible_moves_after = [simulation.get_legal_moves(i) for i in range(simulation.n_snakes)]
    snakes_alive_after = simulation.snakes_alive

    for i in range(len(states_before)):
        if snakes_alive_before[i]:
            replay_buffer.append((states_before[i], moves[i], scores[i], states_after[i], possible_moves_after[i], snakes_alive_after[i]))

    return states_after, possible_moves_after, snakes_alive_after, scores, running

In [14]:
def sample_experiences(batch_size: int):
    idxs = np.random.randint(len(replay_buffer), size=batch_size)
    
    batch = [replay_buffer[idx] for idx in idxs]

    return [
        [experience[field_idx] for experience in batch]
        for field_idx in range(6)
    ]

In [15]:
losses = []


def training_step(buffer_split: float):
    #experiences = sample_experiences(batch_size)
    experiences = replay_buffer.sample(split=buffer_split, batch_size=batch_size)
    states, moves, scores, states_after, possible_moves_after, snakes_alive_after = experiences

    best_next_moves = [[move] for move in get_model_moves(online_model, states_after, possible_moves_after, snakes_alive_after)]
    next_q_values = get_model_scores(target_model, states_after, best_next_moves, snakes_alive_after)
    
    #next_q_values = get_model_scores(target_model, states_after, possible_moves_after, snakes_alive_after)
    runs = 1.0 - np.array(snakes_alive_after)
    
    target_q_values = scores + runs * discount_factor * next_q_values

    X = tf.constant([get_rotated_state(state, move) for state, move in zip(states, moves)])
    
    with tf.GradientTape() as tape:
        predicted_q_values = online_model(X)
        individual_losses = loss_fn(target_q_values, predicted_q_values)
        loss = tf.reduce_mean(individual_losses)

    replay_buffer.update_loss(individual_losses, loss)

    losses.append(loss.numpy())
    grads = tape.gradient(loss, online_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, online_model.trainable_variables))

In [16]:
simulation = Simulation(calculate_score, True)
rewards = deque(maxlen=best_model_window)
replay_buffer.clear()

best_model_rewards = float("-inf")

for episode in range(n_episodes):
    simulation.reset(make_simple_board(np.array([15, 15])), snake_count=2, food_count=1, tail_len=2)

    rewards.append(0)

    states = [simulation.get_snake_view(i, view_type, view_range) for i in range(simulation.n_snakes)]
    possible_moves = [simulation.get_legal_moves(i) for i in range(simulation.n_snakes)]
    snakes_alive = simulation.snakes_alive
    
    epsilon = max(1 - (episode / (n_episodes * 0.8)), 0.005)
    priority_buffer_split = min(episode / (n_episodes * 0.8) * 0.2, 0.2)

    step = 0
    
    while True:
        states, possible_moves, snakes_alive, scores, running = play_one_step(simulation, states, possible_moves, snakes_alive, epsilon)

        rewards[-1] += sum(scores)

        if not running:
            break

        step += 1

    logging.info(f"Episode {episode} played")
    logging.info(f"\tSteps: {step}")
    logging.info(f"\tRewards: {rewards[-1]}")

    if episode > 200:
        curr_model_rewards = sum(rewards)
        if curr_model_rewards > best_model_rewards:
            avg_rewards = curr_model_rewards / best_model_window
            best_model_rewards = curr_model_rewards
            logging.info(f"\tSnaphshot saved (avg. {avg_rewards})")
            
            if model_name:
                online_model.save(f"models/{model_name}_{round(avg_rewards)}_snapshot.keras")

        for _ in range(1):
            training_step(priority_buffer_split)
        
        logger.info(f"\tPriority: {len(replay_buffer.priority_buffer)}")
        
        if episode % 100 == 0:
            target_model.set_weights(online_model.get_weights())
            logging.info("\tTarget model updated")

Episode 0 played
	Steps: 3
	Rewards: -15
Episode 1 played
	Steps: 12
	Rewards: -7
Episode 2 played
	Steps: 12
	Rewards: -10
Episode 3 played
	Steps: 4
	Rewards: -11
Episode 4 played
	Steps: 28
	Rewards: 7
Episode 5 played
	Steps: 8
	Rewards: -7
Episode 6 played
	Steps: 7
	Rewards: -11
Episode 7 played
	Steps: 8
	Rewards: -22
Episode 8 played
	Steps: 1
	Rewards: -20
Episode 9 played
	Steps: 7
	Rewards: -15
Episode 10 played
	Steps: 10
	Rewards: -32
Episode 11 played
	Steps: 25
	Rewards: -26
Episode 12 played
	Steps: 14
	Rewards: -11


2025-10-13 23:47:22.027781: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.16 = (f32[3,64,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[3,32,15,15]{3,2,1,0} %bitcast.316, f32[64,32,5,5]{3,2,1,0} %bitcast.323, f32[64]{0} %bitcast.325), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1_2/convolution" source_file="/home/rs/PycharmProjects/snake-multi/.venv/lib64/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-10-13 23:47:22.070831: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:54

Episode 13 played
	Steps: 23
	Rewards: -23
Episode 14 played
	Steps: 10
	Rewards: -5
Episode 15 played
	Steps: 7
	Rewards: -11
Episode 16 played
	Steps: 5
	Rewards: -11
Episode 17 played
	Steps: 2
	Rewards: -9
Episode 18 played
	Steps: 15
	Rewards: -6
Episode 19 played
	Steps: 16
	Rewards: -15
Episode 20 played
	Steps: 11
	Rewards: -6
Episode 21 played
	Steps: 3
	Rewards: -13
Episode 22 played
	Steps: 10
	Rewards: -9
Episode 23 played
	Steps: 12
	Rewards: -3
Episode 24 played
	Steps: 4
	Rewards: -15
Episode 25 played
	Steps: 5
	Rewards: 2
Episode 26 played
	Steps: 11
	Rewards: 15
Episode 27 played
	Steps: 11
	Rewards: 20
Episode 28 played
	Steps: 36
	Rewards: -2
Episode 29 played
	Steps: 8
	Rewards: -9
Episode 30 played
	Steps: 8
	Rewards: -13
Episode 31 played
	Steps: 14
	Rewards: -15
Episode 32 played
	Steps: 4
	Rewards: -13
Episode 33 played
	Steps: 3
	Rewards: -22
Episode 34 played
	Steps: 7
	Rewards: -22
Episode 35 played
	Steps: 3
	Rewards: -11
Episode 36 played
	Steps: 11
	Rewar

2025-10-13 23:47:24.063402: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.16 = (f32[6,64,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[6,32,15,15]{3,2,1,0} %bitcast.316, f32[64,32,5,5]{3,2,1,0} %bitcast.323, f32[64]{0} %bitcast.325), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1_2/convolution" source_file="/home/rs/PycharmProjects/snake-multi/.venv/lib64/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-10-13 23:47:24.127100: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:54

Episode 63 played
	Steps: 14
	Rewards: -17
Episode 64 played
	Steps: 19
	Rewards: 5
Episode 65 played
	Steps: 6
	Rewards: -7
Episode 66 played
	Steps: 17
	Rewards: 5
Episode 67 played
	Steps: 18
	Rewards: -7
Episode 68 played
	Steps: 15
	Rewards: 6
Episode 69 played
	Steps: 3
	Rewards: -13
Episode 70 played
	Steps: 20
	Rewards: -2
Episode 71 played
	Steps: 10
	Rewards: -14
Episode 72 played
	Steps: 4
	Rewards: -7
Episode 73 played
	Steps: 7
	Rewards: -9
Episode 74 played
	Steps: 16
	Rewards: 8
Episode 75 played
	Steps: 15
	Rewards: 4
Episode 76 played
	Steps: 7
	Rewards: -15
Episode 77 played
	Steps: 4
	Rewards: -15
Episode 78 played
	Steps: 4
	Rewards: -17
Episode 79 played
	Steps: 3
	Rewards: -13
Episode 80 played
	Steps: 13
	Rewards: -17
Episode 81 played
	Steps: 9
	Rewards: 13
Episode 82 played
	Steps: 12
	Rewards: -17
Episode 83 played
	Steps: 10
	Rewards: -9
Episode 84 played
	Steps: 10
	Rewards: -15
Episode 85 played
	Steps: 18
	Rewards: -25
Episode 86 played
	Steps: 9
	Rewards:

...

	Priority: 74
Episode 287 played
	Steps: 28
	Rewards: -21
	Priority: 73
Episode 288 played
	Steps: 5
	Rewards: -8
	Priority: 72
Episode 289 played
	Steps: 8
	Rewards: -1
	Priority: 69
Episode 290 played
	Steps: 9
	Rewards: -19
	Priority: 65
Episode 291 played
	Steps: 10
	Rewards: -11


2025-10-13 23:48:13.721965: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.16 = (f32[20,64,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[20,32,15,15]{3,2,1,0} %bitcast.316, f32[64,32,5,5]{3,2,1,0} %bitcast.323, f32[64]{0} %bitcast.325), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1_2/convolution" source_file="/home/rs/PycharmProjects/snake-multi/.venv/lib64/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-10-13 23:48:13.813644: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:

	Priority: 63
Episode 292 played
	Steps: 18
	Rewards: -25
	Priority: 61
Episode 293 played
	Steps: 18
	Rewards: 2
	Priority: 58
Episode 294 played
	Steps: 27
	Rewards: -21
	Priority: 59
Episode 295 played
	Steps: 9
	Rewards: -13
	Priority: 59
Episode 296 played
	Steps: 26
	Rewards: 8
	Priority: 57
Episode 297 played
	Steps: 17
	Rewards: -27
	Priority: 56
Episode 298 played
	Steps: 15
	Rewards: -21
	Priority: 53
Episode 299 played
	Steps: 11
	Rewards: -23
	Priority: 54
Episode 300 played
	Steps: 4
	Rewards: 2


2025-10-13 23:48:18.342797: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.16 = (f32[17,64,15,15]{3,2,1,0}, u8[0]{0}) custom-call(f32[17,32,15,15]{3,2,1,0} %bitcast.316, f32[64,32,5,5]{3,2,1,0} %bitcast.323, f32[64]{0} %bitcast.325), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1_2/convolution" source_file="/home/rs/PycharmProjects/snake-multi/.venv/lib64/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-10-13 23:48:18.420302: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:

	Priority: 53
	Target model updated
Episode 301 played
	Steps: 7
	Rewards: 11
	Priority: 51
Episode 302 played
	Steps: 6
	Rewards: -13
	Priority: 52
Episode 303 played
	Steps: 15
	Rewards: -9
	Priority: 51
Episode 304 played
	Steps: 6
	Rewards: -13
	Priority: 51
Episode 305 played
	Steps: 4
	Rewards: -11
	Priority: 50
Episode 306 played
	Steps: 9
	Rewards: -7
	Priority: 50
Episode 307 played
	Steps: 8
	Rewards: -13
	Priority: 49
Episode 308 played
	Steps: 3
	Rewards: -13
	Priority: 51
Episode 309 played
	Steps: 4
	Rewards: -18
	Priority: 50
Episode 310 played
	Steps: 4
	Rewards: 2
	Priority: 51
Episode 311 played
	Steps: 14
	Rewards: -17
	Priority: 49
Episode 312 played
	Steps: 7
	Rewards: -5
	Priority: 48
Episode 313 played
	Steps: 3
	Rewards: -13
	Priority: 47
Episode 314 played
	Steps: 9
	Rewards: -5
	Priority: 45
Episode 315 played
	Steps: 9
	Rewards: -18
	Priority: 44
Episode 316 played
	Steps: 4
	Rewards: -15
	Priority: 43
Episode 317 played
	Steps: 6
	Rewards: -19
	Priority: 41


In [17]:
online_model.save(f"models/{model_name}_final.keras")